In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100
import pickle

In [2]:
# df = pd.read_pickle('../data/processed_train.pkl')

In [3]:
# df.shape

In [2]:
from train_classification import DATA_DIR,map_dataset,get_group_dict,load_dict
from utils import fc,create_feats,nm_features
import joblib
from sklearn.preprocessing import RobustScaler

In [7]:
df = pd.read_pickle('../data/processed_train.pkl')

In [8]:
df.shape

(3018000, 58)

In [10]:
df.isnull().sum().sum()

0

In [3]:
config_path = '../configs/lstm-transformer.yaml'

In [4]:
df = pd.read_csv(DATA_DIR + "train.csv")
config = OmegaConf.load(config_path)
pressure_dict = dict(
    zip(df["pressure"].unique().tolist(), range(df["pressure"].nunique()))
)
pressure_dict = {
    v: i for i, v in enumerate(sorted(df["pressure"].unique().tolist()))
}
pressure_reverse_dict = {v: k for k, v in pressure_dict.items()}

# joblib.dump(pressure_reverse_dict, "../pressure_mapper.pkl")
joblib.dump(pressure_reverse_dict, "../sorted_pressure_mapper.pkl")
df["pressure"] = df["pressure"].map(pressure_dict)
df = nm_features(df)
# if config.create_features:
#     df = fc(df)
print(df.shape)
df = df.groupby("breath_id").head(config.seq_len)
print(df.shape)
num_classes = df["pressure"].nunique() 
# config.model.kwargs["output_dim"] = num_classes
num_cols = config.dataset.train.kwargs.numerical_columns
df.fillna(0,inplace=True)
if config.normalization.is_norm:
    scl = StandardScaler()
    print(config.dataset.train.kwargs.numerical_columns)
    for col in config.dataset.train.kwargs.numerical_columns:
        if col != "preds":
            df[col] = scl.fit_transform(df[[col]])


(6036000, 67)
(3018000, 67)
['R', 'C', 'time_step', 'u_in', 'bidc', 'u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 'u_in_lag_4', 'u_in_cumsum', 'u_in_cumsum_lag_1', 'u_in_cumsum_lag_2', 'u_in_cumsum_lag_1-u_in_cumsum_lag_2', 'u_in_cumsum_lag_3', 'u_in_cumsum_lag_4', 'u_in_cummean', 'u_in_cummax', 'next_u_in', 'area', 'area_lag_1', 'area_lag_2', 'area_lead_1', 'area_lead_2', 'area_diff_lag_1', 'area_diff_lead_1', 'u_in_cumsum*time_step', 'u_in_cumsum*time_step_lag_1', 'u_in_cumsum*time_step/c', 'u_in_cumsum*time_step/c_lag_1', 'area/c', 'area/c_lag_1', 'u_out_lag_1', 'time_step*u_out', 'R+C', 'R/C', 'u_in/C', 'u_in/R', 'u_in_cumsum/C', 'u_in_cumsum/R', 'area*R/C', 'u_in_cumsum*R/C', 'u_in_cumsum*R/C_lag_1', 'timestep_diff', 'u_in_diff', 'u_in_pct_change', 'u_in_diff_next', 'u_in_log', 'u_in_cumsum_log', 'u_in_lag_1_is_zero', 'u_in_zero', 'u_in_lead_1', 'maop', 'spike', 'u_in_lag_1_is_zero_cumsum', 'is_max_u_in', 'nki', 'nki2', 'nki3', 'nki4', 'u_in_cummax - u_in']


In [5]:
df.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,R_cat,C_cat,RC_cat,bidc,u_in_lag_1,u_in_lag_2,u_in_lag_3,u_in_lag_4,u_in_cumsum,u_in_cumsum_lag_1,u_in_cumsum_lag_2,u_in_cumsum_lag_1-u_in_cumsum_lag_2,u_in_cumsum_lag_3,u_in_cumsum_lag_4,u_in_cummean,u_in_cummax,next_u_in,area,area_lag_1,area_lag_2,area_lead_1,area_lead_2,area_diff_lag_1,area_diff_lead_1,u_in_cumsum*time_step,u_in_cumsum*time_step_lag_1,u_in_cumsum*time_step/c,u_in_cumsum*time_step/c_lag_1,area/c,area/c_lag_1,u_out_lag_1,time_step*u_out,R+C,R/C,u_in/C,u_in/R,u_in_cumsum/C,u_in_cumsum/R,area*R/C,u_in_cumsum*R/C,u_in_cumsum*R/C_lag_1,timestep_diff,u_in_diff,u_in_pct_change,u_in_diff_next,u_in_log,u_in_cumsum_log,u_in_lag_1_is_zero,u_in_zero,u_in_lead_1,maop,spike,u_in_lag_1_is_zero_cumsum,is_max_u_in,nki,nki2,nki3,nki4,u_in_cummax - u_in
0,1,1,-0.359072,1.394522,-1.685675,-0.614677,0,110,1,2,0,-1.689278,-6.204765,-4.344579,-3.504065,-2.994913,-0.857938,-3.118732,-2.763682,-0.229643,-0.795328,-0.775243,-0.920852,-1.068380,0.424106,-0.683444,-0.665904,-0.648603,-0.697024,-0.704757,-0.566897,0.491921,-0.685961,-0.670334,-0.775451,-0.757956,-0.809349,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.543888,-0.422895,-0.964948,-0.522067,-0.604429,-0.733632,-0.720441,-6.080090,0.032266,-0.108135,-1.812876,-1.064578,-3.458379,-0.67504,-0.67504,0.424106,-1.923327,-0.380129,-0.660098,-0.16961,-0.695780,-1.683736,0.17441,-1.448229,-0.776598
1,2,1,-0.359072,1.394522,-1.597761,0.397359,0,111,1,2,0,-1.602648,0.087395,-4.344579,-3.504065,-2.994913,-0.806998,-0.670142,-2.763682,6.115873,-0.795328,-0.775243,-0.412099,-0.494947,0.656989,-0.679151,-0.665904,-0.648603,-0.686688,-0.689295,-0.490275,0.380639,-0.684250,-0.670334,-0.774619,-0.757956,-0.806976,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.159510,-0.116007,-0.940831,-0.508587,-0.602902,-0.717240,-0.720365,0.261680,1.813803,-0.103604,-0.433589,0.995823,-1.442889,-0.67504,-0.67504,0.656989,-1.866924,-0.380129,-0.660098,-0.16961,-0.652397,-1.597482,0.17441,-1.373675,-0.776598
2,3,1,-0.359072,1.394522,-1.509299,0.625553,0,139,1,2,0,-1.516018,0.202645,0.175893,-3.504065,-2.994913,-0.744623,-0.625088,-0.530394,-0.112886,-0.795328,-0.775243,-0.166038,-0.365648,0.673895,-0.668607,-0.661518,-0.648603,-0.670936,-0.666313,-0.378672,0.282288,-0.678342,-0.668559,-0.771747,-0.757093,-0.801147,-0.790600,-0.524734,-0.560718,0.681566,-0.732796,-0.072840,-0.046809,-0.911302,-0.492083,-0.599152,-0.697168,-0.703649,0.301218,0.433968,-0.108130,-0.061195,1.133688,-0.906754,-0.67504,-0.67504,0.673895,-1.810520,-0.380129,-0.660098,-0.16961,-0.609014,-1.511229,0.17441,-1.299121,-0.776598
3,4,1,-0.359072,1.394522,-1.420404,0.642119,0,194,1,2,0,-1.429389,0.228632,0.258692,0.239309,-2.994913,-0.681419,-0.569921,-0.489302,-0.086679,-0.795088,-0.775243,-0.038844,-0.356262,0.817648,-0.652538,-0.650744,-0.644113,-0.647525,-0.635429,-0.280038,0.143209,-0.668124,-0.662430,-0.766780,-0.754114,-0.792264,-0.784604,-0.524734,-0.560718,0.681566,-0.732796,-0.066548,-0.041786,-0.881379,-0.475358,-0.593436,-0.676829,-0.683180,0.332444,0.061427,-0.108135,-0.279908,1.142732,-0.603956,-0.67504,-0.67504,0.817648,-1.754116,-0.380129,-0.660098,-0.16961,-0.565631,-1.424975,0.17441,-1.224567,-0.776598
4,5,1,-0.359072,1.394522,-1.331024,0.782979,0,201,1,2,0,-1.342759,0.230518,0.277362,0.307875,0.292417,-0.611156,-0.514021,-0.438986,-0.084776,-0.742144,-0.775000,0.065797,-0.276449,0.925109,-0.628654,-0.634324,-0.633084,-0.616063,-0.598535,-0.140558,-0.002983,-0.652634,-0.651829,-0.759250,-0.748961,-0.779062,-0.775466,-0.524734,-0.560718,0.681566,-0.732796,-0.013049,0.000928,-0.848115,-0.456766,-0.584940,-0.654218,-0.662439,0.367400,0.280227,-0.108133,-0.217334,1.215333,-0.373495,-0.67504,-0.67504,0.925109,-1.697712,-0.380129,-0.660098,-0.16961,-0.522247,-1.338721,0.17441,-1.150013,-0.776598


In [6]:
# df = pd.read_pickle("../data/nm_processed_train.pkl")

In [7]:
# df.head()

In [8]:
df.shape

(3018000, 67)

In [9]:
df.to_pickle('../data/nm_processed_train.pkl')

In [11]:
folds = GroupKFold(n_splits=10)
folds = list(folds.split(df, groups=df["breath_id"]))

In [12]:
import pickle

In [13]:
# def map_dataset(data):
#     data.breath_id = data.breath_id.map(
#         dict(zip(data.breath_id.unique().tolist(), range(data.breath_id.nunique())))
#     )
#     data.R = data.R.map(R_MAP)
#     data.C = data.C.map(C_MAP)
#     return data


In [16]:
df.shape

(3018000, 58)

In [17]:
for i in [0]:
    train = df.iloc[folds[i][0]]
    val = df.iloc[folds[i][1]]
    print(train.shape, val.shape)
    train = map_dataset(train)
    val = map_dataset(val)
    train_grp_dict = get_group_dict(train)
    val_grp_dict = get_group_dict(val)
    with open('../data/10_fold_classification_train_grp_dict_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(train_grp_dict, f, pickle.HIGHEST_PROTOCOL)        
    with open('../data/10_fold_classification_grp_dict_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(val_grp_dict, f, pickle.HIGHEST_PROTOCOL)    

(2716200, 58) (301800, 58)


/home/sahil_okcredit_in/miniconda3/envs/torch-py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
100%|██████████| 7545/7545 [00:35<00:00, 214.87it/s]


In [12]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,R_cat,C_cat,RC_cat,bidc,u_in_lag_1,u_in_lag_2,u_in_lag_3,u_in_lag_4,u_in_cumsum,u_in_cumsum_lag_1,u_in_cumsum_lag_2,u_in_cumsum_lag_1-u_in_cumsum_lag_2,u_in_cumsum_lag_3,u_in_cumsum_lag_4,u_in_cummean,u_in_cummax,next_u_in,area,area_lag_1,area_lag_2,area_lead_1,area_lead_2,area_diff_lag_1,area_diff_lead_1,u_in_cumsum*time_step,u_in_cumsum*time_step_lag_1,u_in_cumsum*time_step/c,u_in_cumsum*time_step/c_lag_1,area/c,area/c_lag_1,u_out_lag_1,time_step*u_out,R+C,R/C,u_in/C,u_in/R,u_in_cumsum/C,u_in_cumsum/R,area*R/C,u_in_cumsum*R/C,u_in_cumsum*R/C_lag_1,timestep_diff,u_in_diff,u_in_pct_change,u_in_diff_next,u_in_log,u_in_cumsum_log,u_in_lag_1_is_zero,u_in_zero,u_in_lead_1,maop,spike,u_in_lag_1_is_zero_cumsum,is_max_u_in,nki,nki2,nki3,nki4,u_in_cummax - u_in
0,1,0,-0.359072,1.394522,-1.685675,-0.614677,0,110,1,2,0,-1.689278,-6.204765,-4.344579,-3.504065,-2.994913,-0.857938,-3.118732,-2.763682,-0.229643,-0.795328,-0.775243,-0.920852,-1.068380,0.424106,-0.683444,-0.665904,-0.648603,-0.697024,-0.704757,-0.566897,0.491921,-0.685961,-0.670334,-0.775451,-0.757956,-0.809349,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.543888,-0.422895,-0.964948,-0.522067,-0.604429,-0.733632,-0.720441,-6.080090,0.032266,-0.108135,-1.812876,-1.064578,-3.458379,-0.67504,-0.67504,0.424106,-1.923327,-0.380129,-0.660098,-0.16961,-0.695780,-1.683736,0.17441,-1.448229,-0.776598
1,2,0,-0.359072,1.394522,-1.597761,0.397359,0,111,1,2,0,-1.602648,0.087395,-4.344579,-3.504065,-2.994913,-0.806998,-0.670142,-2.763682,6.115873,-0.795328,-0.775243,-0.412099,-0.494947,0.656989,-0.679151,-0.665904,-0.648603,-0.686688,-0.689295,-0.490275,0.380639,-0.684250,-0.670334,-0.774619,-0.757956,-0.806976,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.159510,-0.116007,-0.940831,-0.508587,-0.602902,-0.717240,-0.720365,0.261680,1.813803,-0.103604,-0.433589,0.995823,-1.442889,-0.67504,-0.67504,0.656989,-1.866924,-0.380129,-0.660098,-0.16961,-0.652397,-1.597482,0.17441,-1.373675,-0.776598
2,3,0,-0.359072,1.394522,-1.509299,0.625553,0,139,1,2,0,-1.516018,0.202645,0.175893,-3.504065,-2.994913,-0.744623,-0.625088,-0.530394,-0.112886,-0.795328,-0.775243,-0.166038,-0.365648,0.673895,-0.668607,-0.661518,-0.648603,-0.670936,-0.666313,-0.378672,0.282288,-0.678342,-0.668559,-0.771747,-0.757093,-0.801147,-0.790600,-0.524734,-0.560718,0.681566,-0.732796,-0.072840,-0.046809,-0.911302,-0.492083,-0.599152,-0.697168,-0.703649,0.301218,0.433968,-0.108130,-0.061195,1.133688,-0.906754,-0.67504,-0.67504,0.673895,-1.810520,-0.380129,-0.660098,-0.16961,-0.609014,-1.511229,0.17441,-1.299121,-0.776598
3,4,0,-0.359072,1.394522,-1.420404,0.642119,0,194,1,2,0,-1.429389,0.228632,0.258692,0.239309,-2.994913,-0.681419,-0.569921,-0.489302,-0.086679,-0.795088,-0.775243,-0.038844,-0.356262,0.817648,-0.652538,-0.650744,-0.644113,-0.647525,-0.635429,-0.280038,0.143209,-0.668124,-0.662430,-0.766780,-0.754114,-0.792264,-0.784604,-0.524734,-0.560718,0.681566,-0.732796,-0.066548,-0.041786,-0.881379,-0.475358,-0.593436,-0.676829,-0.683180,0.332444,0.061427,-0.108135,-0.279908,1.142732,-0.603956,-0.67504,-0.67504,0.817648,-1.754116,-0.380129,-0.660098,-0.16961,-0.565631,-1.424975,0.17441,-1.224567,-0.776598
4,5,0,-0.359072,1.394522,-1.331024,0.782979,0,201,1,2,0,-1.342759,0.230518,0.277362,0.307875,0.292417,-0.611156,-0.514021,-0.438986,-0.084776,-0.742144,-0.775000,0.065797,-0.276449,0.925109,-0.628654,-0.634324,-0.633084,-0.616063,-0.598535,-0.140558,-0.002983,-0.652634,-0.651829,-0.759250,-0.748961,-0.779062,-0.775466,-0.524734,-0.560718,0.681566,-0.732796,-0.013049,0.000928,-0.848115,-0.456766,-0.584940,-0.654218,-0.662439,0.367400,0.280227,-0.108133,-0.217334,1.215333,-0.373495,-0.67504,-0.67504,0.925109,-1.697712,-0.380129,-0.660098,-0.16961,-0.522247,-1.338721,0.17441,-1.150013,-0.776598


In [14]:
val.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,R_cat,C_cat,RC_cat,bidc,u_in_lag_1,u_in_lag_2,u_in_lag_3,u_in_lag_4,u_in_cumsum,u_in_cumsum_lag_1,u_in_cumsum_lag_2,u_in_cumsum_lag_1-u_in_cumsum_lag_2,u_in_cumsum_lag_3,u_in_cumsum_lag_4,u_in_cummean,u_in_cummax,next_u_in,area,area_lag_1,area_lag_2,area_lead_1,area_lead_2,area_diff_lag_1,area_diff_lead_1,u_in_cumsum*time_step,u_in_cumsum*time_step_lag_1,u_in_cumsum*time_step/c,u_in_cumsum*time_step/c_lag_1,area/c,area/c_lag_1,u_out_lag_1,time_step*u_out,R+C,R/C,u_in/C,u_in/R,u_in_cumsum/C,u_in_cumsum/R,area*R/C,u_in_cumsum*R/C,u_in_cumsum*R/C_lag_1,timestep_diff,u_in_diff,u_in_pct_change,u_in_diff_next,u_in_log,u_in_cumsum_log,u_in_lag_1_is_zero,u_in_zero,u_in_lead_1,maop,spike,u_in_lag_1_is_zero_cumsum,is_max_u_in,nki,nki2,nki3,nki4,u_in_cummax - u_in
240,241,0,1.171893,1.394522,-1.685675,-0.619286,0,116,2,2,3,-1.689278,-6.204765,-4.344579,-3.504065,-2.994913,-0.858169,-3.118732,-2.763682,-0.229643,-0.795328,-0.775243,-0.925486,-1.070992,-0.542165,-0.683444,-0.665904,-0.648603,-0.700958,-0.717062,-0.566897,0.563356,-0.685961,-0.670334,-0.775451,-0.757956,-0.809349,-0.793041,-0.524734,-0.560718,1.892657,-0.386222,-0.545639,-0.424292,-0.965057,-0.522128,-0.604429,-0.733707,-0.720441,-6.080090,0.032266,-0.108135,-0.154908,-1.121756,-3.514227,-0.675040,-0.675040,-0.542165,-0.231211,-0.380129,-0.660098,-0.16961,-0.869312,-1.683736,-0.025906,-1.597337,-0.776598
241,242,0,1.171893,1.394522,-1.602460,-0.549463,0,109,2,2,3,-1.602648,0.086870,-4.344579,-3.504065,-2.994913,-0.854671,-0.670346,-2.763682,6.115344,-0.795328,-0.775243,-0.890386,-1.031429,-0.387588,-0.683165,-0.665904,-0.648603,-0.699223,-0.712952,-0.561916,0.536814,-0.685851,-0.670334,-0.775397,-0.757956,-0.809195,-0.793041,-0.524734,-0.560718,1.892657,-0.386222,-0.519120,-0.415823,-0.963401,-0.521757,-0.604181,-0.730892,-0.720441,-0.077272,0.155178,0.152680,-0.298572,-0.538509,-2.944541,1.481395,1.481395,-0.387588,-0.174808,-0.380129,-0.469310,-0.16961,-0.825929,-1.597482,-0.025906,-1.522783,-0.776598
242,243,0,1.171893,1.394522,-1.519103,-0.397997,0,120,2,2,3,-1.516018,0.094821,0.175516,-3.504065,-2.994913,-0.843583,-0.667252,-0.530581,-0.221624,-0.795328,-0.775243,-0.827924,-0.945607,-0.253372,-0.681394,-0.665619,-0.648603,-0.695036,-0.705850,-0.535297,0.492296,-0.685037,-0.670219,-0.775001,-0.757900,-0.808216,-0.792883,-0.524734,-0.560718,1.892657,-0.386222,-0.461592,-0.397451,-0.958151,-0.520584,-0.602607,-0.721972,-0.717571,-0.067073,0.298898,-0.108090,-0.263465,0.028118,-2.234034,-0.675040,-0.675040,-0.253372,-0.118404,-0.380129,-0.469310,-0.16961,-0.782546,-1.511229,-0.025906,-1.448229,-0.776598
243,244,0,1.171893,1.394522,-1.433582,-0.266483,0,150,2,2,3,-1.429389,0.112070,0.181228,0.238997,-2.994913,-0.825905,-0.657445,-0.527758,-0.204229,-0.795328,-0.775243,-0.763637,-0.871089,-0.145246,-0.677123,-0.663810,-0.648311,-0.687802,-0.695388,-0.490651,0.436957,-0.682867,-0.669375,-0.773946,-0.757490,-0.805855,-0.791876,-0.524734,-0.560718,1.892657,-0.386222,-0.411642,-0.381499,-0.949782,-0.518713,-0.598808,-0.707750,-0.708474,0.089059,0.263778,-0.108123,-0.218479,0.305985,-1.744002,-0.675040,-0.675040,-0.145246,-0.062000,-0.380129,-0.469310,-0.16961,-0.739163,-1.424975,-0.025906,-1.373675,-0.776598
244,245,0,1.171893,1.394522,-1.350686,-0.160534,0,180,2,2,3,-1.342759,0.127047,0.193620,0.243728,0.292143,-0.802919,-0.641810,-0.518814,-0.189125,-0.791692,-0.775243,-0.703760,-0.811056,-0.060857,-0.669743,-0.659446,-0.646459,-0.677144,-0.681462,-0.435152,0.374796,-0.678920,-0.667124,-0.772028,-0.756396,-0.801775,-0.789447,-0.524734,-0.560718,1.892657,-0.386222,-0.371401,-0.368648,-0.938900,-0.516280,-0.592245,-0.689258,-0.693972,-0.100321,0.218773,-0.108129,-0.177552,0.470856,-1.392024,-0.675040,-0.675040,-0.060857,-0.005596,-0.380129,-0.469310,-0.16961,-0.695780,-1.338721,-0.025906,-1.299121,-0.776598


In [15]:
a = load_dict('../data/10_fold_NM_grp_dict_fold_0.pkl')

In [18]:
(a[0]['u_in'])

[-0.6192857726075481,
 -0.5494631185982828,
 -0.39799746848507106,
 -0.2664827721971104,
 -0.16053350792836676,
 -0.07784297118676119,
 -0.017892271820586232,
 0.11826812037054318,
 0.2654706723700176,
 0.39243462332202095,
 0.49763025483038453,
 0.5864154352801452,
 0.6540021534629796,
 0.7062981861259136,
 0.6831898249475681,
 0.4577517883479422,
 0.25768889139552165,
 0.0783582244840635,
 -0.07915567401197293,
 -0.2160246927759347,
 -0.32477199918033256,
 -0.41059366109572626,
 -0.3791171604227833,
 -0.30854949623598055,
 -0.2526500553592757,
 -0.21248915397810997,
 -0.18531537792086775,
 -0.16957110815598603,
 -0.16404032104733723,
 -0.16536530653808468,
 -0.1712654657112715,
 -0.18078060547923425,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481,
 -0.6192857726075481]

In [12]:
# a[0]['pressure']

In [22]:
# len(train_grp_dict),len(val_grp_dict)

In [27]:
def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [28]:
len(load_dict('../data/val_grp_dict_fold_0.pkl'))

15090